In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve, train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Read in survey data.
df = pd.read_csv("survey_results_public.csv")

In [ ]:
# Filter out all non-students.
students = df[df['Employment'].str.contains('Student', na=False)]

In [ ]:
# Drop features with mostly missing values.
nalimit = len(students) * 0.55
students = students.dropna(axis='columns', thresh=nalimit)
del students['ResponseId']  # Don't need response ID.

In [ ]:
# Explore the domain of each feature (all unique values).
for feature in students:
    responses = set()

    # Survey responses are semicolon-separated, so they must be unraveled.
    for response in students[feature].dropna():
        responses |= set(str(response).split(';'))

    print(f'{feature} ({len(responses)}): {"; ".join(sorted(responses))}\n')

In [ ]:
# Select data that can either be counted or categorized into features.
countable_data = ['LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith',
                  'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith',
                  'MiscTechHaveWorkedWith', 'ToolsTechHaveWorkedWith']
categorizable_data = ['MainBranch', 'EdLevel', 'Country', 'Age', 'Ethnicity']

# Isolate the IDEs column; this will be used in the output vector.
output = 'NEWCollabToolsHaveWorkedWith'

# Generate features by counting the number of responses to a question.
df1 = students[countable_data].applymap(lambda x: x.count(';') + 1,
                                        na_action='ignore')

# Generate features by encoding distinct survey responses as digits.
df2 = students[categorizable_data].apply(lambda x: x.factorize()[0])

# Generate boolean features based on employment status
# and usage of version control systems (i.e., Git).
s1 = students['Employment'].str.contains('Employed')
s2 = students['VersionControlSystem'].map(lambda x: x != "I don't use one",
                                          na_action='ignore')

# Generate additional numeric features based on the number of operating
# systems used by the respondent and how many years they have coded.
s3 = students['OpSysPersonal use'] + ';' + students['OpSysProfessional use']
s3 = s3.map(lambda x: len(set(x.split(';'))), na_action='ignore').fillna(1)
s3.rename("OperatingSystems", inplace=True)
s4 = students['YearsCode'].replace('Less than 1 year', 0.5)
s4 = s4.replace('More than 50 years', 50.5)

In [ ]:
# Combine each generated feature into the design matrix.
X = pd.concat([df1, df2, s1, s2, s3, s4], axis=1).fillna(0).astype(float)

# Classify respondents by their usage of VS Code in the output vector.
y = students[output].str.contains('Visual Studio Code').fillna(0).astype(int)

In [ ]:
# Split data into training (60%), validation (20%), and test (20%) sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6,
                                                    random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5,
                                                random_state=42)

In [ ]:
# Apply feature scaling to the training data.
scaler = StandardScaler().fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)

In [ ]:
# Use logistic regression to determine a decision boundary.
logmodel = LogisticRegression().fit(X_train, y_train)

In [ ]:
# Initial accuracy test.
y_pred = logmodel.predict(X_test)
logmodel.score(X_test, y_test)